In [1]:
%load_ext autoreload
%autoreload 2
import sys
# sys.path.append("/Users/fdangelo/PycharmProjects/myRBM")
import numpy as np 
import scipy.constants as cs
import deepdish as dd
from sklearn.preprocessing import Binarizer
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
data = dd.io.load('../data/ising/ising_data_L32.h5')
data.keys()

dict_keys(['1.0', '2.2', '3.0'])

In [3]:
data_T1 = data['1.0']
data_T2_2 = data['2.2']
data_T3 = data['3.0']

In [4]:
binarizer = Binarizer(threshold=0)
data_T1_binary = np.array([binarizer.fit_transform(slice) for slice in data_T1])
data_T2_2_binary = np.array([binarizer.fit_transform(slice) for slice in data_T2_2])
data_T3_binary = np.array([binarizer.fit_transform(slice) for slice in data_T3])

In [5]:
magn_T1 = np.abs(np.array([np.average(slice) for slice in data_T1]))
magn_T2_2 = np.abs(np.array([np.average(slice) for slice in data_T2_2]))
magn_T3= np.abs(np.array([np.average(slice) for slice in data_T3]))

In [6]:
# create a new plot with default tools, using figure
p = figure(plot_width=600, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(np.arange(len(magn_T1)),magn_T1, size=7, line_color="navy", fill_color="green", fill_alpha=0.5, legend = 'T = 1.0')
p.circle(np.arange(len(magn_T1)),magn_T2_2, size=7, line_color="navy", fill_color="yellow", fill_alpha=0.5, legend = 'T = 2.2')
p.circle(np.arange(len(magn_T1)),magn_T3, size=7, line_color="navy", fill_color="blue", fill_alpha=0.5, legend = 'T = 3.0')

p.yaxis.axis_label = "Magnetization"
p.xaxis.axis_label = "Sample"
p.legend.location = "bottom_right"
show(p) # show the results

In [7]:
mean_magn_T1 = np.average(magn_T1)
mean_magn_T2_2 = np.average(magn_T2_2)
mean_magn_T3= np.average(magn_T3)
std_magn_T1 = np.std(magn_T1)
std_magn_T2_2 = np.std(magn_T2_2)
std_magn_T3 = np.std(magn_T3)
mean_magn_temp = [mean_magn_T1,mean_magn_T2_2,mean_magn_T3]
std_magn_temp = [std_magn_T1,std_magn_T2_2,std_magn_T3]
temp = [2.0/2.269,2.269/2.269,2.5/2.269]
T = np.linspace(0.1,2.3,100)
teor_magn = np.nan_to_num(np.power((1-np.power(np.sinh(2/(T)),-4)),0.125))

/home/user/emil.steiner/.local/share/miniconda3/envs/RBM/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in power
  # This is added back by InteractiveShellApp.init_path()


In [8]:
# create a new plot with default tools, using figure
from bokeh.models import ColumnDataSource, Whisker
p = figure(plot_width=600, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(temp,mean_magn_temp, size=7, line_color="navy", fill_color="green", fill_alpha=0.5, legend = 'Simulated conf')
p.line(T/2.269, teor_magn,line_color="red", line_width=3, line_alpha=0.6, legend='Theoretical curve' )
source_error = ColumnDataSource(data=dict(base=temp, lower=np.asarray(mean_magn_temp)-np.asarray(std_magn_temp), upper=np.asarray(mean_magn_temp)+np.asarray(std_magn_temp)))
p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)

p.yaxis.axis_label = "Magnetization"
p.xaxis.axis_label = "T/Tcritic"
p.legend.location = "bottom_left"
show(p) # show the results